# Тестовое задание

In [1]:
import pandas as pd
import numpy as np
import nltk
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
import string

In [28]:
count_vect = CountVectorizer(max_df = 0.7, min_df = 0.05, ngram_range=(1, 3))
tfidf_vect = TfidfVectorizer(max_df = 0.7, min_df = 0.05, ngram_range=(1, 3))
porter_stemmer = nltk.stem.PorterStemmer()
tok = count_vect.build_tokenizer()
clf = LogisticRegression(solver='liblinear')

## Cкачиваем файл с данными

In [451]:
#!wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Electronics_5.json.gz
#или
#!wget https://drive.google.com/open?id=1DwwTyIn5zW7KTxp4bONHtKNa6DlT9PUj

## Если необходимо, преобразуем JSON к CSV

In [452]:
#df = pd.read_json ('reviews_Electronics_5.json', lines=True )
#df.to_csv ('Electronics_5.csv', index = None)

# Предобработка

In [3]:
dat = pd.read_csv("Electronics_5.csv")[["asin","reviewText", "overall"]]

## Добавляем бинрную оценку

In [4]:
dat["bin"] = dat.overall > 3

## Отрежем кусок даты

In [5]:
dat_t, dat = dat, dat[0:5001]

## Натравим на необработанные данные

In [29]:
dat_count_deft = count_vect.fit_transform(dat.reviewText.apply(lambda x: np.str_(x)))
dat_tfidf_deft = tfidf_vect.fit_transform(dat.reviewText.apply(lambda x: np.str_(x)))


## Приводим к базовой форме

In [7]:
dat["words"] = list(map(lambda elem: (" ").join(map(lambda s: (porter_stemmer.stem(s)), elem)).lower().translate(str.maketrans('', '', string.punctuation)), map(lambda wrds: tok(wrds), dat.reviewText.apply(lambda x: np.str_(x)))))

/home/denis/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


## Можно воспользоваться анализатором, который предоставляют векторайзеры

In [82]:
# anlz = count_vect.build_analyzer()
# dat.words = list(map(lambda s: (" ").join(anlz(s)), dat.reviewText))

## Натравим на обработанные данные

In [30]:
dat_count = count_vect.fit_transform(dat.words)
dat_tfidf = tfidf_vect.fit_transform(dat.words)

## Проверим, что получилось

In [31]:
dat

,asin,reviewText,overall,bin,words
0,0528881469,We got this GPS for my husband who is an (OTR)...,5,True,we got thi gp for my husband who is an otr ove...
1,0528881469,"I'm a professional OTR truck driver, and I bou...",1,False,profession otr truck driver and bought tnd 700...
2,0528881469,"Well, what can I say. I've had this unit in m...",3,False,well what can say ve had thi unit in my truck ...
3,0528881469,"Not going to write a long review, even thought...",2,False,not go to write long review even thought thi u...
4,0528881469,I've had mine for a year and here's what we go...,1,False,ve had mine for year and here what we got it t...
...,...,...,...,...,...
4996,B00001WRSJ,Pro:Great sound. Fairly effective sound isola...,3,False,pro great sound fairli effect sound isol well ...
4997,B00001WRSJ,I did not know that Sony made headphones of th...,5,True,did not know that soni made headphon of thi qu...
4998,B00001WRSJ,"Just like the title of my review says, I've ha...",5,True,just like the titl of my review say ve had the...
4999,B00001WRSJ,I love these headphones. The sound is superb. ...,4,True,love these headphon the sound is superb bass t...


## Разобьем вектора на выборки

In [32]:
count_train_mtrx, count_test_mtrx, count_train_ans, count_test_ans = train_test_split(
    dat_count, dat.bin, test_size=0.2)
#dat_count.toarray()

In [33]:
tfidf_train_mtrx, tfidf_test_mtrx, tfidf_train_ans, tfidf_test_ans = train_test_split(
    dat_tfidf, dat.bin, test_size=0.2)
#dat_tfidf.toarray()

In [34]:
count_train_mtrx_dft, count_test_mtrx_dft, count_train_ans_dft, count_test_ans_dft = train_test_split(
    dat_count_deft, dat.bin, test_size=0.2)

In [35]:
tfidf_train_mtrx_dft, tfidf_test_mtrx_dft, tfidf_train_ans_dft, tfidf_test_ans_dft = train_test_split(
    dat_tfidf_deft, dat.bin, test_size=0.2)

## Обучим модели и посмотрим их точность

In [36]:
clf.fit(count_train_mtrx, count_train_ans)
cout_pred = clf.predict(count_test_mtrx)
accuracy_score(cout_pred, count_test_ans)

0.8121878121878122

In [37]:
clf.fit(tfidf_train_mtrx, tfidf_train_ans)
tfidf_pred = clf.predict(tfidf_test_mtrx)
accuracy_score(tfidf_pred, tfidf_test_ans)

0.8241758241758241

In [38]:
clf.fit(count_train_mtrx_dft, count_train_ans_dft)
cout_pred = clf.predict(count_test_mtrx_dft)
accuracy_score(cout_pred, count_test_ans_dft)

0.8161838161838162

In [39]:
clf.fit(tfidf_train_mtrx_dft, tfidf_train_ans_dft)
tfidf_pred = clf.predict(tfidf_test_mtrx_dft)
accuracy_score(tfidf_pred, tfidf_test_ans_dft)

0.8251748251748252

In [50]:
param_grid = {
    'C': np.arange(0.5, 3, 0.1),
    'penalty': ['l1', 'l2']
}
search = GridSearchCV(clf, param_grid, cv=5)
search.fit(dat_count, dat.bin)
print(search.best_params_)

{'C': 0.5, 'penalty': 'l1'}


In [53]:
accuracy_score(dat.bin, search.best_estimator_.predict(dat_count))

0.8484303139372126

In [42]:
param_grid = {
    'C': np.arange(0.5, 3, 0.1),
    'penalty': ['l1', 'l2']
}
search = GridSearchCV(clf, param_grid, cv=5)
search.fit(dat_tfidf, dat.bin)
print(search.best_params_)

{'C': 1.8999999999999997, 'penalty': 'l1'}


In [49]:
accuracy_score(dat.bin, search.best_estimator_.predict(dat_tfidf))

0.8524295140971806

In [54]:
param_grid = {
    'C': np.arange(0.5, 3, 0.1),
    'penalty': ['l1', 'l2']
}
search = GridSearchCV(clf, param_grid, cv=5)
search.fit(dat_count_deft, dat.bin)
print(search.best_params_)

{'C': 0.5, 'penalty': 'l2'}


In [55]:
accuracy_score(dat.bin, search.best_estimator_.predict(dat_count_deft))

0.8542291541691661

In [56]:
param_grid = {
    'C': np.arange(0.5, 3, 0.1),
    'penalty': ['l1', 'l2']
}
search = GridSearchCV(clf, param_grid, cv=5)
search.fit(dat_tfidf_deft, dat.bin)
print(search.best_params_)

{'C': 1.4, 'penalty': 'l2'}


In [57]:
accuracy_score(dat.bin, search.best_estimator_.predict(dat_tfidf_deft))

0.8460307938412317